In [ ]:
# Simulación de Ventas y Proyecciones Futuras
# Este script simula proyecciones de ventas futuras utilizando arrays aleatorios de Numpy.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Simular proyecciones de ventas futuras usando arrays aleatorios de Numpy. 
# Crear un DataFrame vacío para almacenar los datos de la simulación
Simulacion_ventas = {
    "ID_Tienda": pd.Series (),
    "Producto": pd.Series (),
    "Cantidad_Stock": pd.Series (),
    "Cantidad_Vendida": pd.Series (),
    "Precio_Unitario": pd.Series (),
    "Costo_produccion": pd.Series (),
    "Ventas_Totales": pd.Series (),
    "Ganancia_Neta": pd.Series (),
    "Fecha_Venta": pd.Series (),
    }
df = pd.DataFrame(Simulacion_ventas)
print (df.head())

In [ ]:
# Crear un inventario de productos y sus costos de producción, precios de venta y producción mensual
# Definir el inventario de productos y sus costos de producción, precios de venta y producción
inventario = {
    "productos": pd.Series(),
    "Costo_produccion": pd.Series(),
    "Precio_venta": pd.Series(),
    "Producción_mensual": pd.Series(),
}
# Solicitar al usuario la cantidad de tiendas y productos que maneja
Tiendas = int(input("\n Ingrese la cantidad de tiendas que maneja: "))
# Solicitar al usuario los datos de cada producto
cantidad = int(input("\n Ingrese la cantidad de productos que maneja: "))
# Se llena el inventario con los datos de los productos
for i in range(cantidad):
    inventario["productos"].loc[i] = input(f"Ingrese el nombre del producto {i+1}: ")
    inventario["Costo_produccion"].loc[i] = float(input(f"Ingrese el costo de producción del producto {i+1}: "))
    inventario["Precio_venta"].loc[i] = float(input(f"Ingrese el precio de venta del producto {i+1}: "))
    inventario["Producción_mensual"].loc[i] = int(input(f"Ingrese la producción mensual del producto {i+1}: "))
inventario_df = pd.DataFrame(inventario)


In [ ]:
# Solicitar al usuario los costos fijos mensuales, tipo de simulación, cantidad de datos a simular e inflación
fijos = float(input("Ingrese los costos fijos mensuales (ejemplo: 1000.0): "))
tipo = int(input("Defina el tipo de simulación que desea realizar (Lineal:1 ,Exponencial:2): "))
simulaciones = int(input("\n Escriba La cantidad de datos que desea simular: "))
inflacion = (float(input("Ingrese el porcentaje de inflación mensual (ejemplo: 0% \rhasta 20%): ")))/3000
ventas_diarias = 1
fecha = pd.to_datetime("2025-01-01")  # Fecha de inicio de la simulación


In [ ]:
# Validación del tipo de simulación
if tipo == 1:
    pendiente = float(input("Ingrese el factor de pendiente (ejemplo: -10.0 a 10.0): "))
elif tipo == 2:
    fact_exponencial = float(input("Ingrese el factor exponencial (ejemplo: 1 a 5): "))
# Según el tipo de simulación, se ajusta los datos de ventas
for i in range(simulaciones):
    df.loc[i, "ID_Tienda"] = np.random.randint(1, Tiendas + 1)
    df.loc[i, "Producto"] = np.random.choice(inventario["productos"])
    df.loc[i, "Precio_Unitario"] = df.loc[i, "Producto"].map(inventario_df.set_index("productos")["Precio_venta"])
    costo_base = df.loc[i, "Producto"].map(inventario_df.set_index("productos")["Costo_produccion"])
    df.loc[i, "Fecha_Venta"] = (pd.to_datetime("2025-01-01") + pd.Timedelta(days=(i//ventas_diarias))).strftime("%Y-%m-%d")
    if df.loc[i, "Fecha_Venta"].month != fecha.month:
        df.loc[i, "Cantidad_Stock"] = inventario_df["Producción_mensual"].loc[inventario_df["productos"] == df.loc[i, "Producto"]].values[0]
    fecha = pd.to_datetime(df.loc[i, "Fecha_Venta"])
    df.loc[i, "Costo_produccion"] = costo_base * ((1 + inflacion) ** i)
    df.loc[i, "Costo_produccion"] = df.loc[i, "Producto"].map(inventario_df.set_index("productos")["Costo_produccion"])
    if tipo == 1:
        if (pendiente <= 1.0) and (pendiente >= -10.0):
            df.loc[i, "Cantidad_Vendida"] = int(np.random.randint((20 + (i * pendiente)),(40 + (i * pendiente))))
            if (20 + (i * pendiente)) < 1:
                upper = max(0, (40 + (i * pendiente)))
                if upper > 0:
                    df.loc[i, "Cantidad_Vendida"] = int(np.random.randint(0, upper))
                else:
                    df.loc[i, "Cantidad_Vendida"] = 0
        elif pendiente == 0.0:
            df.loc[i, "Cantidad_Vendida"] = int(np.random.randint(15,40))
        elif pendiente > 0.0 and pendiente <= 10.0:
            df.loc[i, "Cantidad_Vendida"] = int(np.random.randint((20 + (i * pendiente)),(40 + (i * pendiente))))
        else:
            print (f"El valor de pendiente dado {pendiente}, se encuentra fuera del rango establecido")
    elif tipo == 2:
        if fact_exponencial < 1 or fact_exponencial > 5:
            print(f"El valor de factor exponencial dado {fact_exponencial}, se encuentra fuera del rango establecido")
        else:
            min_val = int(np.trunc(20 + np.power(fact_exponencial, i)))
            max_val = int(np.trunc(40 + np.power(fact_exponencial, i)))
            if max_val > min_val:
                df.loc[i, "Cantidad_Vendida"] = int(np.random.randint(min_val, max_val))
            else:
                df.loc[i, "Cantidad_Vendida"] = min_val
    else:
        print ("Se seleccionó un tipo de simulación invalido")
    df.loc[i, "Fecha_Venta"] = (pd.to_datetime("2025-01-01") + pd.Timedelta(days=i)).strftime("%Y-%m-%d")
df["Ventas_Totales"] = df["Cantidad_Vendida"] * df["Precio_Unitario"]
print(df)
# Calcular la ganancia neta
df["Ganancia_Neta"] = df["Ventas_Totales"] - (inventario_df.set_index("productos").loc[df["Producto"],
     "Costo_produccion"] * df["Cantidad_Vendida"]) - fijos/30

# Crear el eje de tiempo (puede ser el índice o la columna Fecha_Venta)
# Gráfica independiente de Ventas Totales (Simuladas)
# Crear el eje de tiempo y los datos de ventas totales y ganancias simuladas a partir de df
tiempo_df = pd.to_datetime(df["Fecha_Venta"])
ventas_totales_df = pd.to_numeric(df["Ventas_Totales"], errors='coerce')
ganancia_neta_df = pd.to_numeric(df["Ganancia_Neta"], errors='coerce')  # Nueva línea

plt.figure(figsize=(10, 5))
plt.plot(tiempo_df, ventas_totales_df, label="Ventas Totales (Simuladas)", color='blue')
plt.plot(tiempo_df, ganancia_neta_df, label="Ganancia Neta", color='orange')  # Nueva línea
plt.xlabel("Tiempo")
plt.ylabel("Valor")
plt.title("Ventas Totales y Ganancia Neta Simuladas")
plt.legend()
plt.show()

# Gráfica independiente de Cantidad Vendida (Simulada)
tiempo_df = pd.to_datetime(df["Fecha_Venta"])
cantidad_vendida_df = pd.to_numeric(df["Cantidad_Vendida"], errors='coerce')

plt.figure(figsize=(10, 5))
plt.plot(tiempo_df, cantidad_vendida_df, label="Cantidad Vendida (Simulada)", color='green')
plt.xlabel("Tiempo")
plt.ylabel("Cantidad Vendida")
plt.title("Cantidad Vendida Simulada")
plt.legend()
plt.show()

# Grafica independiente de Precio Unitario (Simulada)
tiempo_df = pd.to_datetime(df["Fecha_Venta"])
precio_unitario_df = pd.to_numeric(df["Precio_Unitario"], errors='coerce') 

plt.figure(figsize=(10, 5))
plt.plot(tiempo_df, precio_unitario_df, label="Precio Unitario (Simulada)", color='red')
plt.xlabel("Tiempo")
plt.ylabel("Precio Unitario")
plt.title("Precio Unitario Simulada")
plt.legend()
plt.show()  